In [ ]:
from process_tweets import get_data
from rnn import MyModel
from train_validate import (train, validate)

import copy
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader

In [ ]:
# hyperparameters
train_batch_size = 250
test_batch_size = 250
hidden_size = 64
embedding_size = 12
EPOCH = 50
train_file="../data/train_en.tsv"
test_file="../data/dev_en.tsv"
device="cpu"

train_dataset, test_dataset, alphabet, longest_sent, train_x, test_x = get_data(train_file, test_file)
train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=True)

# Define inputs
num_classes = 2

model = MyModel(len(alphabet),longest_sent,embedding_size,hidden_size,num_classes,device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters())

## training loop
best = 0.0
best_cm = None
best_model = None
acc_per_epoch = []
for epoch in range(EPOCH):
    # train loop
    train(epoch, train_loader, model, optimizer, criterion)

    # validation loop
    acc, cm = validate(epoch, test_loader, model, criterion)
    acc_per_epoch.append(acc.detach().cpu().numpy() )

    if acc > best:
        best = acc
        best_cm = cm
        best_model = copy.deepcopy(model)

print('Best Prec @1 Acccuracy: {:.4f}'.format(best))
per_cls_acc = best_cm.diag().detach().numpy().tolist()
for i, acc_i in enumerate(per_cls_acc):
    print("Accuracy of Class {}: {:.4f}".format(i, acc_i))


plt.figure()
plt.plot(range(EPOCH), acc_per_epoch)
plt.title("loss curve")
plt.xlabel('epoch')
plt.ylabel('validation accuracy')
plt.show()